# Einführung in RapidFuzz

Dieses Notebook demonstriert die Kernfunktionen von RapidFuzz für Fuzzy-String-Matching.

## Installation

Falls RapidFuzz noch nicht installiert ist:

In [1]:
%pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


## Import

In [12]:
from rapidfuzz import fuzz, process

# Beispielstrings
text1 = "RapidFuzz ist großartig"
text2 = "Rapid Fuzz ist wirklich großartig"

## Grundlegende String-Ähnlichkeit

`fuzz.ratio(a, b)`:
- Berechnet die Ähnlichkeit zweier Strings basierend auf dem Levenshtein-Abstand (Einfügen, Löschen, Ersetzen von Zeichen).
- Ergebnis ist ein Score zwischen 0 und 100, wobei 100 perfekte Übereinstimmung bedeutet.

`fuzz.partial_ratio(a, b)`:
- Nützlich, wenn ein String Teil des anderen ist.
- Beispiel: "RapidFuzz" vs. "RapidFuzz ist toll" → Partial Ratio erkennt, dass "RapidFuzz" im längeren String enthalten ist.

In [13]:
ratio_score = fuzz.ratio(text1, text2)
partial_score = fuzz.partial_ratio(text1, text2)

print(f"Ratio: {ratio_score}")
print(f"Partial Ratio: {partial_score}")

Ratio: 82.14285714285714
Partial Ratio: 75.0


## Token-basierte Ähnlichkeiten

Token-basierte Methoden teilen Strings in Wörter (Tokens) auf und vergleichen diese auf unterschiedliche Weise. Das hilft besonders bei Wortreihenfolge oder kleinen Umstellungen.

`fuzz.token_sort_ratio(a, b)`:
- Sortiert die Wörter alphabetisch und vergleicht die sortierten Strings.
- Gut, wenn die Wörter dieselben sind, aber in unterschiedlicher Reihenfolge stehen.

`fuzz.token_set_ratio(a, b)`:
- Vergleicht die Schnittmenge der Wörter und die Differenzmenge.
- Sehr nützlich, wenn ein String mehr oder weniger zusätzliche Wörter enthält.

In [14]:
token_sort = fuzz.token_sort_ratio(text1, text2)
token_set = fuzz.token_set_ratio(text1, text2)

print(f"Token Sort Ratio: {token_sort}")
print(f"Token Set Ratio: {token_set}")

Token Sort Ratio: 67.85714285714286
Token Set Ratio: 72.22222222222223


## Mehrere Kandidaten vergleichen (`extract` / `extractOne`)

Wenn man einen Suchbegriff mit einer Liste von Möglichkeiten vergleichen möchte, helfen die process-Funktionen:

`process.extract(query, choices, scorer=...)`
- Vergleicht query mit allen Einträgen in choices.
- Liefert eine Liste von Tupeln (Choice, Score, Index) sortiert nach Score absteigend.
- So kann man die besten Matches oder mehrere relevante Ergebnisse auf einmal sehen.

`process.extractOne(query, choices, scorer=...)`
- Gibt nur das beste Matching zurück.
- Praktisch, wenn man z.B. einen einzigen Kandidaten automatisch auswählen möchte.

In [15]:
choices = ["RapidFuzz ist super", "Python ist toll", "Rapid Fuzz ist großartig"]

# Extract alle Matches
all_matches = process.extract("RapidFuzz großartig", choices, scorer=fuzz.token_set_ratio)
print("Alle Matches:")
print(all_matches)

# Bestes Match
best_match = process.extractOne("RapidFuzz großartig", choices, scorer=fuzz.token_set_ratio)
print("\nBestes Match:")
print(best_match)

Alle Matches:
[('Rapid Fuzz ist großartig', 69.76744186046511, 2), ('RapidFuzz ist super', 64.28571428571428, 0), ('Python ist toll', 17.647058823529406, 1)]

Bestes Match:
('Rapid Fuzz ist großartig', 69.76744186046511, 2)


## Anwendungsszenario: Duplikaterkennung

Wir prüfen eine Liste von Produktnamen auf mögliche Duplikate.

In [23]:
products = ["iPhone 14 Pro", "iPhone14Pro", "Samsung Galaxy S23", "Samsung Galaxy S23 Ultra"]

for i, prod in enumerate(products):
    for j in range(i+1, len(products)):
        score = fuzz.token_set_ratio(prod, products[j])
        if score > 45:
            print(f"Mögliche Duplikate: '{prod}' und '{products[j]}' (Score: {score})")

Mögliche Duplikate: 'iPhone 14 Pro' und 'iPhone14Pro' (Score: 50.0)
Mögliche Duplikate: 'Samsung Galaxy S23' und 'Samsung Galaxy S23 Ultra' (Score: 100.0)
